#### 1. Importing Libraries

In [1]:
import sys
import requests
import bs4
import json
import re
import time
from nltk.tokenize import sent_tokenize
import re
import os
from urllib.parse import quote

#### 2. Extracting hindi WikiPages for all scientists dump whereever it exists

In [2]:
person_list = []
for i in range(1,25):
    with open('./Data/Wikidata_JSON_businessperson/businessperson_dump_part'+str(i)+'.json') as f:
        new_list = json.load(f)
        #print(len(new_list))
        person_list = person_list + new_list

In [3]:
print(len(person_list))

4800


In [5]:
#person_list[1]['labels']['hi']['value']

In [6]:
person_list[1]['id']

'Q185'

In [4]:
person_name = quote(person_list[1]['labels']['hi']['value'])
print(person_name)
res = requests.get('https://hi.wikipedia.org/wiki/' + person_list[1]['labels']['hi']['value'])
print(res)

%E0%A4%B2%E0%A5%87%E0%A4%B0%E0%A5%80%20%E0%A4%B8%E0%A5%87%E0%A4%82%E0%A4%97%E0%A4%B0
<Response [404]>


In [7]:
t0 = time.time()
articles = {}
count_persons = 0
output_dir = 'Data/Hindi_Wikipages_businessperson'
os.makedirs(output_dir, exist_ok=True)

for person in person_list:
    try:
        # Check if the 'hi' (Hindi) label exists for the person
        if 'hi' in person['labels']:
            # Properly encode the Wikipedia page title in Hindi
            person_name = quote(person['labels']['hi']['value'])
            res = requests.get('https://hi.wikipedia.org/wiki/' + person_name)  # Hindi Wikipedia URL

            # Check if the page exists
            if res.status_code != 404:
                print(person['labels']['hi']['value'])
                # Parse the page content in Hindi
                wiki = bs4.BeautifulSoup(res.text, "html.parser")
                elems = wiki.select('p')  # Get all paragraphs

                # Extract text from <p> tags and clean it
                article = []
                for elem in elems:
                    text = elem.getText().replace('\n', '').lower()
                    text = text.replace('\ufeff', ' ')
                    text = re.sub(r'\[\d+\]', '', text.strip())  # Remove reference numbers like [1], [2]
                    article.extend(sent_tokenize(text))  # Use sentence tokenization

                # Remove any empty sentences from the article
                article = [sent.strip() for sent in article if sent.strip()]

                # Store the cleaned article in the articles dictionary
                if article:
                    articles[person['id']] = article

            count_persons += 1

            # Save articles every 200 persons
            if count_persons % 200 == 0:
                partition_num = str(count_persons // 200)
                with open(f'{output_dir}/hi_articles_part{partition_num}.json', 'w', encoding='utf-8') as fout:
                    json.dump(articles, fout, indent=4, ensure_ascii=False)  # Save in UTF-8 for Hindi characters
                
                # Reset articles for the next batch
                articles = {}

                # Print progress
                print(f"Checkpoint {partition_num} reached, JSON dumps saved | Time Elapsed: {time.time() - t0:.2f} seconds")

        #else:
            #print(f"No Hindi label for person ID {person['id']}")

    except Exception as e:
        print(f"Error processing person ID {person['id']}: {e}")

# Save remaining articles (if any) after the loop finishes
if articles:
    partition_num = str((count_persons // 200) + 1)  # Calculate the next partition number
    with open(f'{output_dir}/hi_articles_part{partition_num}.json', 'w', encoding='utf-8') as fout:
        json.dump(articles, fout, indent=4, ensure_ascii=False)
    print(f"Final save for remaining articles: {len(articles)} articles saved in part {partition_num}")

# Final timing and completion message
t1 = time.time()
total = t1 - t0
print(f"Total time elapsed: {total:.2f} seconds")

जिमी वेल्स
जॉर्ज वॉकर बुश
जूलियन असांजे
जेन्ना जेम्सन
ओसामा बिन लादेन
बज़ एल्ड्रिन
अर्नोल्ड श्वार्ज़नेगर
माइकल जैक्सन
50 सॅण्ट
स्नूप दॉग
आन्द्रे अगासी
थॉमस अल्वा एडीसन
स्टीवन स्पिलबर्ग
जॉन मेनार्ड कीन्स
कैथरीन, कैम्ब्रिज की डचेस
एर्टन सेना
आर्थर नेविल चाम्बरलेन
ड्वेन जॉनसन
सेरेना विलियम्स
हैरी एस ट्रूमैन
निक्की हेली
ली म्युंग बाक
कान्ये वेस्ट
गेरार्ड पिके
अनुशेह अंसारी
अल्बर्ट गोर
जोसेफिन कोक्रेन
डोनाल्ड ट्रम्प
दिमित्री मेदवेदेव
जिमी कार्टर
बर्नार्ड अरनॉल्ट
जेनिफ़र एनिस्टन
अलेक्ज़ांडर ग्राहम बेल
ग्विनिथ पाल्ट्रो
हर्बर्ट क्लार्क हूवर
वारन गैमेलियल हार्डिंग
गुल्येल्मो मार्कोनी
जॉर्ज लूकस
नफ्ताली बेनेट
सैंड्रा बुलक
मारिया कैरी
खदीजा बिन्त खुवायलद
कैटी पेरी
रीज विदरस्पून
जेसिका एल्बा
पेरिस हिल्टन
डिक चेनी
दान क़्वेयल:
गिनी रोमेटी
जॉन अगेइकुम कुफूर
रुपर्ट मर्डोक
येलिपेड्डी पद्मश्री वारियर
ओपरा विनफ्रे
शेख मोहम्मद बिन राशिद अल मकतूम
यिंगलक चिनावाट
पेट्रो पोरोशेंको
हेनरी किसिंजर
हेल्मा ओरोज
बोरिस बेकर
स्टीवन सीगल
सेलिना गोमेज़
पीटर ड्रकर
एरिक एमर्सन श्मिट
रॉबर्ट मेटकॉफ
मेरी वुलस्टोनक्राफ़्ट
ल

In [8]:
print(total)
print(len(articles))

97.76759171485901
25
